# Tool Calling

Key Benefits

- **Enhanced Functionality**: Perform complex tasks like data retrieval, calculations, or command execution.
- **Structured Outputs**: Ensure outputs match predefined schemas for seamless data processing.
- **Dynamic Decision-Making**: Invoke appropriate tools based on conversation context for intelligent interactions.
- **Seamless Integration**: Enable smooth information flow across tools and components in workflows.
- **Improved User Experience**: Provide accurate, relevant responses using real-time data or calculations.
- **Flexibility & Customization**: Create custom tools to meet specific needs, enhancing application adaptability.

In [1]:
from langchain_openai import ChatOpenAI

In [2]:
llm_model="gpt-4o-mini"
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [3]:
llm.invoke("How will the weather be in Bangalore today?")

AIMessage(content="I'm unable to provide real-time weather updates. For the latest weather information in Bangalore, I recommend checking a reliable weather website or app.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 16, 'total_tokens': 43, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BdWWbeeD8Nd3JTbvIMtlkvk6RZOT9', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8a27a8e2-5430-4879-88bb-44c80c245e35-0', usage_metadata={'input_tokens': 16, 'output_tokens': 27, 'total_tokens': 43, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
from langchain_core.tools import tool

#defining different tools

# @tool decorator creates LangChain tools

@tool
def fake_weather_api(city: str) -> str:
    """
    Check the weather in a specified city.

    Args:
        city (str): The name of the city where you want to check the weather.

    Returns:
        str: A description of the current weather in the specified city.
    """
    return "Sunny, 22°C"  #this could be an external API, like Google Weather


@tool
def outdoor_seating_availability(city: str) -> str:
    """
    Check if outdoor seating is available at a specified restaurant in a given city.

    Args:
        city (str): The name of the city where you want to check for outdoor seating availability.

    Returns:
        str: A message stating whether outdoor seating is available or not.
    """
    return "Outdoor seating is available."


tools = [fake_weather_api, outdoor_seating_availability]

In [5]:
llm_with_tools = llm.bind_tools(tools) # allows llms to access tools

In [6]:
result = llm_with_tools.invoke("How will the weather be in Bangalore today?")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bjLkW4ZMFCQM5zdIHrsnt1U9', 'function': {'arguments': '{"city":"Bangalore"}', 'name': 'fake_weather_api'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 157, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BdWbUxVTJHSdu4DRhRrYpI1RoF2zu', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7c79a35c-92c9-466c-98dd-ae3387dacddc-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Bangalore'}, 'id': 'call_bjLkW4ZMFCQM5zdIHrsnt1U9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 157, 'output_tokens': 16, 'total_tokens': 173, 'input_to

In [7]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

messages = [
    HumanMessage(
        "How will the weather be in Bangalore today? Do you still have seats outdoor available?"
    )
]

result = llm_with_tools.invoke(messages)  #this only decides the tool to call
messages.append(result)

In [8]:
result.tool_calls

[{'name': 'fake_weather_api',
  'args': {'city': 'Bangalore'},
  'id': 'call_5lkWIb917nz2Cmi1Tz4FCvT9',
  'type': 'tool_call'},
 {'name': 'outdoor_seating_availability',
  'args': {'city': 'Bangalore'},
  'id': 'call_YZMmWow4Hvdz9HTMoS2LdxxH',
  'type': 'tool_call'}]

In [9]:
tool_mapping = {
    "fake_weather_api": fake_weather_api,
    "outdoor_seating_availability": outdoor_seating_availability,
}

In [10]:
from langchain_core.messages import ToolMessage

# Iterate over each tool call in the result's tool_calls list
for tool_call in result.tool_calls:

    # Retrieve the corresponding tool from the tool_mapping dictionary using the tool name (case insensitive)
    tool = tool_mapping[tool_call["name"].lower()]

    # Invoke the selected tool with the provided arguments and store the output
    tool_output = tool.invoke(tool_call["args"])

    # Create a ToolMessage object with the tool's output and the unique ID of the tool call
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [11]:
messages

[HumanMessage(content='How will the weather be in Bangalore today? Do you still have seats outdoor available?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5lkWIb917nz2Cmi1Tz4FCvT9', 'function': {'arguments': '{"city": "Bangalore"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_YZMmWow4Hvdz9HTMoS2LdxxH', 'function': {'arguments': '{"city": "Bangalore"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 165, 'total_tokens': 216, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BdWbxhNU2tYjK6aV4gRvG9rnduSr4', 'service_tier': 'default', 'finish_re

In [14]:
llm_with_tools.invoke(messages)

AIMessage(content='The weather in Bangalore today is sunny with a temperature of 22°C. Additionally, outdoor seating is available at restaurants in the area.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 244, 'total_tokens': 272, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-Bd5gOxiDmnZlkC9e43HhPZrysMx4p', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f689c75f-009d-4fb0-90bb-a1ef9f28e020-0', usage_metadata={'input_tokens': 244, 'output_tokens': 28, 'total_tokens': 272, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Math Tool

In [19]:
from langchain.agents import load_tools
from langchain_openai import ChatOpenAI
from langchain.agents import AgentType

In [20]:
llm = ChatOpenAI(model="gpt-4o-mini")

tool_names = ["llm-math"]
tools = load_tools(tool_names, llm=llm)
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question

### React Template

ReAct is a general paradigm that combines reasoning and acting with LLMs. ReAct prompts LLMs to generate verbal reasoning traces and actions for a task. This allows the system to perform dynamic reasoning to create, maintain, and adjust plans for acting while also enabling interaction to external environments (e.g., Wikipedia) to incorporate additional information into the reasoning. The figure below shows an example of ReAct and the different steps involved to perform question answering.

![](react.webp)

In [21]:
from langchain.agents import initialize_agent

agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, # Zero shot react prompt
                         verbose=True,
                         max_iterations=3)

In [22]:
agent.invoke(input="How many members does the A Team have?")



> Entering new AgentExecutor chain...
To answer the question about how many members the A Team has, I need to consider what "A Team" refers to. Typically, the A Team refers to a fictional group of characters from a television show. I will recall that the original A Team consists of four main members.

Final Answer: The A Team has 4 members.

> Finished chain.


{'input': 'How many members does the A Team have?',
 'output': 'The A Team has 4 members.'}

In [24]:
agent.invoke(input="What is 100 multiplied by 100? What is the square root of that number?")



> Entering new AgentExecutor chain...
To solve the question, I need to first calculate the product of 100 multiplied by 100, and then I will find the square root of that result.

Action: Calculator
Action Input: 100 * 100
Observation: Answer: 10000
Thought:Now that I have the product, which is 10,000, I need to calculate the square root of this number.

Action: Calculator
Action Input: 10000 ** 0.5
Observation: Answer: 100.0
Thought:I now know the final answer
Final Answer: 100.0

> Finished chain.


{'input': 'What is 100 multiplied by 100? What is the square root of that number?',
 'output': '100.0'}